In [13]:
import sys
print(sys.executable)

/Users/everestkc/miniforge3/envs/py313/bin/python3.13


In [14]:
import pandas as pd

In [15]:
import pickle

In [16]:
import seaborn as sns
import matplotlib.pyplot as plt

In [18]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [45]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
print("No. of columns =",len(df.columns))

No. of columns = 19


In [46]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
print("Standard Deviation of duration =",df.duration.std())

Standard Deviation of duration = 42.59435124195458


In [47]:
length_before_dropping = len(df)
df = df[(df.duration >= 1) & (df.duration <= 60)]
length_after_dropping = len(df)
print("Fraction =",(length_after_dropping/length_before_dropping)*100 )


categorical = ['PULocationID', 'DOLocationID']

df[categorical] = df[categorical].astype(str)

Fraction = 98.1220282212598


In [44]:
train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

print(f"Feature matrix shape: {X_train}")

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

Feature matrix shape: <Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6018346 stored elements and shape (3009173, 515)>
  Coords	Values
  (0, 43)	1.0
  (0, 325)	1.0
  (1, 148)	1.0
  (1, 456)	1.0
  (2, 149)	1.0
  (2, 461)	1.0
  (3, 227)	1.0
  (3, 299)	1.0
  (4, 237)	1.0
  (4, 266)	1.0
  (5, 38)	1.0
  (5, 325)	1.0
  (6, 45)	1.0
  (6, 409)	1.0
  (7, 108)	1.0
  (7, 304)	1.0
  (8, 147)	1.0
  (8, 328)	1.0
  (9, 6)	1.0
  (9, 303)	1.0
  (10, 225)	1.0
  (10, 404)	1.0
  (11, 178)	1.0
  (11, 494)	1.0
  (12, 45)	1.0
  :	:
  (3009160, 328)	1.0
  (3009161, 155)	1.0
  (3009161, 494)	1.0
  (3009162, 203)	1.0
  (3009162, 306)	1.0
  (3009163, 50)	1.0
  (3009163, 325)	1.0
  (3009164, 242)	1.0
  (3009164, 401)	1.0
  (3009165, 54)	1.0
  (3009165, 326)	1.0
  (3009166, 64)	1.0
  (3009166, 482)	1.0
  (3009167, 46)	1.0
  (3009167, 401)	1.0
  (3009168, 203)	1.0
  (3009168, 266)	1.0
  (3009169, 233)	1.0
  (3009169, 271)	1.0
  (3009170, 150)	1.0
  (3009170, 273)	1.0
  (3009171, 237)	1.0
  (3009171, 

7.649262960840922

In [50]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [51]:
# df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [ ]:
len(df_train), len(df_val)

In [ ]:
# df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
# df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']
val_dicts = df_val[categorical].to_dict(orient='records')

In [53]:
# categorical = ['PULocationID', 'DOLocationID']

# dv = DictVectorizer()

train_dicts = df[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [55]:
target = 'duration'
y_train = df[target].values
y_val = df_val[target].values

In [56]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.811797467593667